In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
import sys
sys.path.append('../')
import API_keys


prompt_template = "Bạn là một trợ lí thông minh chuyên hỗ trợ trong thư viện, giúp người dùng tìm kiếm sách\
                    Công việc của bạn là phân tích ý định của người dùng thông qua yêu cầu của họ\
                Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)
llm = OpenAI()
chain = prompt | llm


In [ ]:

chain.invoke("your adjective here")

# SELF QUERYING

In [2]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
import sys
sys.path.append('../')
import API_keys

docs = [
    Document(
        page_content="Tâm bệnh học",
        metadata={"type": "Y học - sức khỏe", "keywords": "Điều trị, Tâm bệnh học", "published_year": 2013},
    ),
    Document(
        page_content="Giáo trình can thiệp sớm cho trẻ khuyết tật",
        metadata={"type": "Khoa học xã hội", "keywords": "Can thiệp sớm, Giáo dục, Trẻ khuyết tật", "published_year": 2015},
    ),
    Document(
        page_content="Tâm lý học giao tiếp",
        metadata={"type": "Khoa học xã hội", "keywords": "Giao tiếp, Tâm lý học", "published_year": 2016},
    ),
    Document(
        page_content="Kỹ thuật cháy",
        metadata={"type": "Khoa học tự nhiên", "keywords": "Điều trị, Tâm bệnh học", "author": "Hoàng Ngọc Đồng (ch.b); Phạm Duy Vũ", "published_year": 2019},
    ),
    Document(
        page_content="CẤU TẠO CỦA PHỨC CHẤT",
        metadata={"type": "Ngôn ngữ", "keywords": "Điều trị, Tâm bệnh học", "author": "Lục Thị Thu Hoài", "published_year": 2024},
    ),
]
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

In [3]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="type",
        description="The kind of book",
        type="string",
    ),
    AttributeInfo(
        name="keywords",
        description="The keywords that can related to the book",
        type="integer",
    ),
    AttributeInfo(
        name="author",
        description="The authors of the book",
        type="string",
    ),
    AttributeInfo(
        name="published_year",
        description="The year of publication",
        type="integer",
    ),
]
document_content_description = "Name of the book"
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

In [10]:
# This example specifies a query and composite filter
retriever.invoke(
    "cuốn sách mà có tác giả là Lục Thị Hoài Thu"
)

[Document(page_content='Tâm lý học giao tiếp', metadata={'keywords': 'Giao tiếp, Tâm lý học', 'type': 'Khoa học xã hội'}),
 Document(page_content='Giáo trình can thiệp sớm cho trẻ khuyết tật', metadata={'keywords': 'Can thiệp sớm, Giáo dục, Trẻ khuyết tật', 'type': 'Khoa học xã hội'}),
 Document(page_content='Tâm bệnh học', metadata={'keywords': 'Điều trị, Tâm bệnh học', 'type': 'Y học - sức khỏe'}),
 Document(page_content='CẤU TẠO CỦA PHỨC CHẤT', metadata={'author': 'Lục Thị Thu Hoài', 'keywords': 'Điều trị, Tâm bệnh học', 'type': 'Ngôn ngữ'})]

In [10]:
# This example specifies a query and a filter
retriever.invoke("list 2 movies with animate genre")

[Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993})]

In [14]:
metadata_field_info = {
    "content": "Lyrics of a song",
    "attributes": {
        "artist": {
            "type": "string",
            "description": "Name of the song artist"
        },
        "length": {
            "type": "integer",
            "description": "Length of the song in seconds"
        },
        "genre": {
            "type": "string",
            "description": "The song genre, one of 'pop', 'rock' or 'rap'"
        }
    }
}

In [4]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

In [6]:
print(prompt.format(query="the book was publish after 2020"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [16]:
query_constructor.invoke("type y tế")

StructuredQuery(query='y tế', filter=None, limit=None)

In [6]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=ChromaTranslator(),
)

In [21]:
retriever.invoke("tác giả Lê Thị Thu Hoài")

[Document(page_content='Tâm lý học giao tiếp', metadata={'keywords': 'Giao tiếp, Tâm lý học', 'published_year': 2016, 'type': 'Khoa học xã hội'}),
 Document(page_content='Tâm bệnh học', metadata={'keywords': 'Điều trị, Tâm bệnh học', 'published_year': 2013, 'type': 'Y học - sức khỏe'}),
 Document(page_content='CẤU TẠO CỦA PHỨC CHẤT', metadata={'author': 'Lục Thị Thu Hoài', 'keywords': 'Điều trị, Tâm bệnh học', 'published_year': 2024, 'type': 'Ngôn ngữ'}),
 Document(page_content='Giáo trình can thiệp sớm cho trẻ khuyết tật', metadata={'keywords': 'Can thiệp sớm, Giáo dục, Trẻ khuyết tật', 'published_year': 2015, 'type': 'Khoa học xã hội'})]

In [1]:
import sys
sys.path.append('../')
import API_keys
from methods import *
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.storage import InMemoryStore, LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.text_splitter import RecursiveCharacterTextSplitter

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap= 20, separators=[" {'"])
child_splitter = RecursiveCharacterTextSplitter(
                separators=["\n","\n\n" "\\n", "\\n\\n", '",', '. '],
                chunk_size=120,
                chunk_overlap=20,
                length_function=len,
                is_separator_regex=False,
            )

db = Chroma(collection_name="split_parents", persist_directory='child', embedding_function=OpenAIEmbeddings(chunk_size=1))
fs1 = LocalFileStore("parent")
parent_store = create_kv_docstore(fs1)
fs2 = LocalFileStore("grandparent")
grandparent_store = create_kv_docstore(fs2)

In [5]:
retriever = CustomParentDocumentRetriever(
                vectorstore=db,
                docstore=parent_store,
                docstore2=grandparent_store,
                parent_splitter=parent_splitter
            )

In [32]:
doc = Document(page_content='Phạm Xuân Ái', metadata={'MSSV': 20134011})

In [33]:
retriever.docstore2.mset([('ai', doc)])

In [34]:
retriever.docstore2.mget(['ai'])

[Document(page_content='Phạm Xuân Ái', metadata={'MSSV': 20134011})]

In [7]:
import json
json_file = '/home/xuanai/LLM/Bot/Knowledge/Books/Json/sample.json'
jq_schema = 'book_infos'
documents = json.loads(Path(json_file).read_text())[jq_schema]

In [13]:
documents[0]

{'Tên sách': 'Sổ tay hướng dẫn sử dụng dịch vụ Thư viện',
 'Loại sách': 'UTE. Thông tin -Thư viện',
 'ID': 1,
 'Keyword': ['Từ khóa: Thư viện, Sổ tay, Dịch vụ'],
 'Mô tả': 'Nhằm góp phần phục vụ tốt nhất cho người dùng trong quá trình sử dụng dịch vụ thư viện, Tổ tư vấn viên đã biên soạn cuốn “Sổ tay tư vấn và hướng dẫn sử dụng thư viện” cung cấp những thông tin bao quát và chi tiết về Thư viện, giúp quá trình tìm kiếm và tiếp cận nguồn tài nguyên thông tin của người dùng trở nên nhanh chóng, chính xác và dễ dàng nhất đáp ứng cho người dạy và người học những tài liệu để nghiên cứu, giảng dạy và học tập.',
 'Vị trí': 'Kệ số 1',
 'Nội dung đầu sách': '/Knowledge/Books/PDF/UTE. Thông tin -Thư viện/final_so_tay_thu_vien_nh23_24_8223.pdf'}